<h1>                             Identifying Business opportunities in the city of London <h1>

<h3> The purpose of this project is to identify setting up of a new business opportunity in a promising 
    neighborhood of London city.Setting up a new business will be finalized after careful analysis of all neighborhoods.

In this project, I will start by identifying all the boroughs and neighborhoods of London City.
After all the boroughs have been identified, we will focus the attention of the project in a specific 
neighborhood of a borough for identifying the people traffic, trends and business preference of customers. <h3>

<h3> Importing all packages <h3>

In [83]:

!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


<h4> Fetching and importing London borough data from the web. <h4>

In [84]:
URL = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')
df_list = []

<h3> Fetching Latitude and Longitude details <h3>

In [85]:
for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data1 = data[0]
    data8 = data[8]    
    try:
        borough_name = data1.get_text()
        borough_name = borough_name.split('[')
        borough_name = borough_name[0]
        borough_name = borough_name.strip()
        
        ll = data8.get_text()
        ll = ll.split('/')
        lat_long = ll[2]
        lat_long = lat_long.split('(')
        lat_long = lat_long[0]
        lat_long = lat_long.split(';')
        latitude = lat_long[0]
        latitude = latitude.strip()
        longitude = lat_long[1]
        longitude = longitude.strip()
        longitude = longitude.replace(u'\ufeff', '')
        latitude = float(latitude)
        longitude = float(longitude)

#       Append the borough name, latitude and logitude in a list
        df_list.append((borough_name, latitude, longitude))
    except IndexError:pass

In [86]:
df_boroughs = pd.DataFrame(df_list, columns=['Borough', 'Latitude' , 'Longitude'])

In [87]:
df_boroughs.shape

(32, 3)

In [88]:
df_boroughs.head()

,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198


In [89]:
df_boroughs.dtypes

Borough       object
Latitude     float64
Longitude    float64
dtype: object

<h4> As discussed previously, I am interested in identifying business opportunties for an entrepreneur in London.
    
To narrow down on a specific neighbourhood, I have identified 'CAMDEN' to be unbeatable in its proximity, location, diversity and buzz.
    
It is a very famous and popular area with tourists, teenagers and punks.Hence we will now drill down and elaborate on this locality. <h4>

In [90]:
df_boroughs.loc[df_boroughs['Borough'] == 'Camden']

,Borough,Latitude,Longitude
5,Camden,51.529,-0.1255


<h4> Using Geopy Library to fetch cordinates for London City <h4>

In [91]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'London, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


<h4> Plot a map of the London City highlighting the boroughs <h4>

In [92]:
import folium 
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df_boroughs['Latitude'], df_boroughs['Longitude'], df_boroughs['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
fill_opacity=0.7).add_to(map_london)  
map_london

<h4>Reading Latitudes and longitudes of all neighbourhoods in Camden borough<h4>

In [93]:
from urllib.request import urlopen
import re
URL = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

codes = []
areas_list = []
href_links_list = []
for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data0 = data[0]
    area_name = data0.text

    data1 = data[1]
    data1 = data1.text
    borough = data1.split('[')
    borough_name = borough[0]
    data5 = data[5]
    code = data5.text
    code = code.strip()
    
    if borough_name == 'Camden':
        codes.append(code)
        areas_list.append((borough_name,area_name,code))

                
for link in soup.findAll('a', attrs={'href': re.compile("^https://tools.wmflabs.org")}):
            htext = link.text
            if htext in codes:
                hlink = link.get('href')
                href_links_list.append((htext, hlink))

<h4> Convert the list to a dataframe. <h4>

In [94]:
df_areas = pd.DataFrame(areas_list, columns=['Borough', 'Area', 'Code'])

In [95]:
df_areas.columns

Index(['Borough', 'Area', 'Code'], dtype='object')

In [96]:
df_areas.shape

(17, 3)

<h4> Building a dataframe from list of href links <h4>

In [97]:
df_links = pd.DataFrame(href_links_list, columns=['Code','href'])

In [98]:
df_links.columns

Index(['Code', 'href'], dtype='object')

In [99]:
df_links.shape

(18, 2)

In [100]:
df_links

,Code,href
0,TQ285875,https://tools.wmflabs.org/geohack/en/51.571733...
1,TQ273845,https://tools.wmflabs.org/geohack/en/51.545045...
2,TQ305825,https://tools.wmflabs.org/geohack/en/51.526342...
3,TQ295845,https://tools.wmflabs.org/geohack/en/51.544545...
4,TQ281844,https://tools.wmflabs.org/geohack/en/51.543965...
5,TQ293816,https://tools.wmflabs.org/geohack/en/51.518529...
6,TQ257853,https://tools.wmflabs.org/geohack/en/51.552593...
7,TQ285855,https://tools.wmflabs.org/geohack/en/51.553760...
8,TQ265855,https://tools.wmflabs.org/geohack/en/51.554211...
9,TQ285875,https://tools.wmflabs.org/geohack/en/51.571733...


<h4> Merge the Area and HREF dataframes <h4>

In [101]:
cols = df_links.columns.difference(df_areas.columns)

In [102]:
cols

Index(['href'], dtype='object')

In [103]:
df_areas_links = pd.concat([df_areas, df_links[cols]], axis=1)

In [104]:
df_areas_links.shape

(18, 4)

In [105]:
df_areas_links

,Borough,Area,Code,href
0,Camden,Belsize Park,TQ273845,https://tools.wmflabs.org/geohack/en/51.571733...
1,Camden,Bloomsbury,TQ305825,https://tools.wmflabs.org/geohack/en/51.545045...
2,Camden,Camden Town,TQ295845,https://tools.wmflabs.org/geohack/en/51.526342...
3,Camden,Chalk Farm,TQ281844,https://tools.wmflabs.org/geohack/en/51.544545...
4,Camden,Fitzrovia,TQ293816,https://tools.wmflabs.org/geohack/en/51.543965...
5,Camden,Frognal,TQ257853,https://tools.wmflabs.org/geohack/en/51.518529...
6,Camden,Gospel Oak,TQ285855,https://tools.wmflabs.org/geohack/en/51.552593...
7,Camden,Hampstead,TQ265855,https://tools.wmflabs.org/geohack/en/51.553760...
8,Camden,Highgate,TQ285875,https://tools.wmflabs.org/geohack/en/51.554211...
9,Camden,Holborn,TQ305815,https://tools.wmflabs.org/geohack/en/51.571733...


<h4> Removing blank rows <h4>

In [106]:
df_areas_links = df_areas_links.dropna(how='any')

In [107]:
df_areas_links

,Borough,Area,Code,href
0,Camden,Belsize Park,TQ273845,https://tools.wmflabs.org/geohack/en/51.571733...
1,Camden,Bloomsbury,TQ305825,https://tools.wmflabs.org/geohack/en/51.545045...
2,Camden,Camden Town,TQ295845,https://tools.wmflabs.org/geohack/en/51.526342...
3,Camden,Chalk Farm,TQ281844,https://tools.wmflabs.org/geohack/en/51.544545...
4,Camden,Fitzrovia,TQ293816,https://tools.wmflabs.org/geohack/en/51.543965...
5,Camden,Frognal,TQ257853,https://tools.wmflabs.org/geohack/en/51.518529...
6,Camden,Gospel Oak,TQ285855,https://tools.wmflabs.org/geohack/en/51.552593...
7,Camden,Hampstead,TQ265855,https://tools.wmflabs.org/geohack/en/51.553760...
8,Camden,Highgate,TQ285875,https://tools.wmflabs.org/geohack/en/51.554211...
9,Camden,Holborn,TQ305815,https://tools.wmflabs.org/geohack/en/51.571733...


<h4>Identify/Fetch the geo co-ordinates of all areas of CAMDEN borough <h4>

In [108]:
geo_codes = []
for row in df_areas_links.itertuples():
    url = row.href
    code = row.Code
    res = requests.get(url).text
    soup1 = BeautifulSoup(res,'lxml')
    
    for lat in soup1.find('span',{'class':'latitude'}):
        latitude = lat
        latitude = float(latitude)
            
    for long in soup1.find('span',{'class':'longitude'}):    
        longitude = long
        longitude = float(longitude)
        
    geo_codes.append((code, latitude, longitude))

print(geo_codes)

[('TQ273845', 51.571734, -0.147219), ('TQ305825', 51.545045, -0.165609), ('TQ295845', 51.526342, -0.120229), ('TQ281844', 51.544545, -0.133901), ('TQ293816', 51.543966, -0.154115), ('TQ257853', 51.51853, -0.137848), ('TQ285855', 51.552593, -0.188385), ('TQ265855', 51.55376, -0.147949), ('TQ285875', 51.554212, -0.176781), ('TQ305815', 51.571734, -0.147219), ('TQ285845', 51.517355, -0.120599), ('TQ282838', 51.544774, -0.148314), ('TQ295825', 51.538551, -0.152893), ('TQ305815', 51.526572, -0.134636), ('TQ305825', 51.517355, -0.120599), ('TQ266842', 51.526342, -0.120229), ('TQ255855', 51.542507, -0.175806)]


<h4> Build a geo-code dataframe <h4>

In [109]:
df_geo_codes = pd.DataFrame(geo_codes, columns=['Code','Latitude','Longitude'])

In [110]:
df_geo_codes

,Code,Latitude,Longitude
0,TQ273845,51.571734,-0.147219
1,TQ305825,51.545045,-0.165609
2,TQ295845,51.526342,-0.120229
3,TQ281844,51.544545,-0.133901
4,TQ293816,51.543966,-0.154115
5,TQ257853,51.518530,-0.137848
6,TQ285855,51.552593,-0.188385
7,TQ265855,51.553760,-0.147949
8,TQ285875,51.554212,-0.176781
9,TQ305815,51.571734,-0.147219


<h4> Merge Neighborhoods and Geocodes Dataframe <h4>

In [136]:
df_areas.columns

Index(['Borough', 'Area', 'Code'], dtype='object')

In [137]:
df_areas.shape

(17, 3)

In [138]:
df_geo_codes.columns

Index(['Code', 'Latitude', 'Longitude'], dtype='object')

In [114]:
df_geo_codes.shape

(17, 3)

In [139]:
cols = df_geo_codes.columns.difference(df_areas.columns)

In [140]:
cols

Index(['Latitude', 'Longitude'], dtype='object')

In [141]:
Camden_borough = pd.concat([df_areas, df_geo_codes[cols]], axis=1)

In [159]:
Camden_borough

,Borough,Neighborhood,Latitude,Longitude
0,Camden,Belsize Park,51.571734,-0.147219
1,Camden,Bloomsbury,51.545045,-0.165609
2,Camden,Camden Town,51.526342,-0.120229
3,Camden,Chalk Farm,51.544545,-0.133901
4,Camden,Fitzrovia,51.543966,-0.154115
5,Camden,Frognal,51.518530,-0.137848
6,Camden,Gospel Oak,51.552593,-0.188385
7,Camden,Hampstead,51.553760,-0.147949
8,Camden,Highgate,51.554212,-0.176781
9,Camden,Holborn,51.571734,-0.147219


<h4> Rename column 'Area' to 'Neighborhood' <h4>

In [149]:
Camden_borough = Camden_borough.rename(columns={'Area' :'Neighborhood'})

In [161]:
Camden_borough.columns

Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [160]:
Camden_borough.columns

Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [162]:
Camden_borough

,Borough,Neighborhood,Latitude,Longitude
0,Camden,Belsize Park,51.571734,-0.147219
1,Camden,Bloomsbury,51.545045,-0.165609
2,Camden,Camden Town,51.526342,-0.120229
3,Camden,Chalk Farm,51.544545,-0.133901
4,Camden,Fitzrovia,51.543966,-0.154115
5,Camden,Frognal,51.518530,-0.137848
6,Camden,Gospel Oak,51.552593,-0.188385
7,Camden,Hampstead,51.553760,-0.147949
8,Camden,Highgate,51.554212,-0.176781
9,Camden,Holborn,51.571734,-0.147219


In [199]:
Camden_borough.dtypes

Borough            object
Neighborhood       object
Latitude          float64
Longitude         float64
Cluster Labels      int32
dtype: object

In [200]:
address = 'Camden, London'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Camden are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Camden are 51.5423045, -0.1395604.


<h4>Plotting a map for all the neighborhoods of CAMDEN borough. <h4>

In [166]:
# create map of Camden using latitude and longitude values
map_Camden = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Camden_borough['Latitude'], Camden_borough['Longitude'], Camden_borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Camden)  
    
map_Camden

<h4> Exploring neighborhoods in Camden <h4>
<h4> Starting with the first neighborhood <h4>
    

In [167]:
Camden_borough.loc[0, 'Neighborhood']

'Belsize Park'

In [168]:
Camden_borough.loc[0]

Borough               Camden
Neighborhood    Belsize Park
Latitude             51.5717
Longitude          -0.147219
Name: 0, dtype: object

<h4>Fetching the longitude and latitude details for Belzine Park. <h4>

In [169]:
neighborhood_latitude = Camden_borough.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Camden_borough.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = Camden_borough.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Belsize Park are 51.571734, -0.147219.


<h4>  Fetching the top 100 places of Camden Neighborhood within 500KM radius <h4>

In [170]:
CLIENT_ID = 'SMJLAVSC0BOEV15BGVTWDOQXE0FTLKM2QMFGEQ1HNROUKAHL' # your Foursquare ID
CLIENT_SECRET = 'IBJ3VOUVX2OGD1BZ1INQIIT0JM425XZ1W34QA3K52VRB0QUZ' # your Foursquare Secret
VERSION = '20180604'

In [171]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret=\
       {}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=SMJLAVSC0BOEV15BGVTWDOQXE0FTLKM2QMFGEQ1HNROUKAHL&client_secret=       IBJ3VOUVX2OGD1BZ1INQIIT0JM425XZ1W34QA3K52VRB0QUZ&v=20180604&ll=51.571734,-0.147219&radius=500&limit=100'

<h4> Segregating the Venues by Categories <h4>

In [172]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h4>JSON Data Cleansing and format it to a dataframe<h4>

In [173]:
results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,The Duke's Head,Pub,51.570502,-0.146158
1,The Red Lion & Sun,Gastropub,51.572621,-0.149911
2,The Gatehouse,Pub,51.571418,-0.150015
3,Waterlow Park,Park,51.569016,-0.145041
4,GAIL's Bakery,Bakery,51.571120,-0.148992


In [174]:
nearby_venues.shape

(27, 4)

In [175]:
nearby_venues['name'].value_counts()

Strada                                    1
The Wrestlers                             1
The Flask                                 1
The Old Crown                             1
Kipling's                                 1
Lauderdale House                          1
GAIL's Bakery                             1
George Michael Memorial                   1
The Red Lion & Sun                        1
Le Pain Quotidien                         1
High Tea of Highgate                      1
Fish Fish                                 1
The Angel Inn                             1
The Duke's Head                           1
PizzaExpress                              1
Prince of Wales                           1
Waterlow Park                             1
Costa Coffee                              1
Cocoro - Sushi & Japanese Delicatessen    1
Côte                                      1
Zelas Drinks & Deli                       1
The Bull                                  1
Highgate Pantry                 

In [88]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


<h4> Explore the CAMDEN borough neighbourhood. <h4>

In [177]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, 
            VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, lat, lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                             'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

<h4>Execute the above function for each neighborhood in CAMDEN.<h4>

In [178]:
Camden_venues = getNearbyVenues(names=Camden_borough['Neighborhood'],
                                   latitudes=Camden_borough['Latitude'],
                                   longitudes=Camden_borough['Longitude']
                                  )

In [179]:
Camden_venues.shape

(953, 7)

In [180]:
Camden_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Belsize Park,51.571734,-0.147219,The Duke's Head,51.570502,-0.146158,Pub
1,Belsize Park,51.571734,-0.147219,The Red Lion & Sun,51.572621,-0.149911,Gastropub
2,Belsize Park,51.571734,-0.147219,The Gatehouse,51.571418,-0.150015,Pub
3,Belsize Park,51.571734,-0.147219,Waterlow Park,51.569016,-0.145041,Park
4,Belsize Park,51.571734,-0.147219,GAIL's Bakery,51.571120,-0.148992,Bakery


<h4>Display venues for all neighborhoods.<h4>

In [182]:
Camden_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Belsize Park,27,27,27,27,27,27
Bloomsbury,33,33,33,33,33,33
Camden Town,100,100,100,100,100,100
Chalk Farm,22,22,22,22,22,22
Fitzrovia,80,80,80,80,80,80
Frognal,95,95,95,95,95,95
Gospel Oak,33,33,33,33,33,33
Hampstead,18,18,18,18,18,18
Highgate,42,42,42,42,42,42


<h4> Identifying the unique categories for all venues. <h4>

In [183]:
print('There are {} uniques categories.'.format(len(Camden_venues['Venue Category'].unique())))

There are 185 uniques categories.


<h4>Analyze all neighborhoods <h4>

In [184]:
# one hot encoding
Camden_onehot = pd.get_dummies(Camden_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Camden_onehot['Neighborhood'] = Camden_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Camden_onehot.columns[-1]] + list(Camden_onehot.columns[:-1])
Camden_onehot = Camden_onehot[fixed_columns]

Camden_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Belsize Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Belsize Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Belsize Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Belsize Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Belsize Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [185]:
Camden_onehot.shape

(953, 186)

<h4> Grouping rows by neighborhood.Taking the mean of the frequency of occurrence of each category <h4>

In [186]:
Camden_grouped = Camden_onehot.groupby('Neighborhood').mean().reset_index()
Camden_grouped.head()

,Neighborhood,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Belsize Park,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,...,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
1,Bloomsbury,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,...,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
2,Camden Town,0.0,0.0,0.0,0.0,0.0,0.0100,0.0,0.0000,0.0,...,0.0000,0.0,0.02,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
3,Chalk Farm,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,...,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
4,Fitzrovia,0.0,0.0,0.0,0.0,0.0,0.0125,0.0,0.0125,0.0,...,0.0125,0.0,0.00,0.0,0.0,0.0,0.0,0.0125,0.0,0.0


In [187]:
Camden_grouped.columns

Index(['Neighborhood', 'African Restaurant', 'American Restaurant', 'Aquarium',
       'Arcade', 'Argentinian Restaurant', 'Art Gallery', 'Art Museum',
       'Arts & Crafts Store', 'Arts & Entertainment',
       ...
       'Vegetarian / Vegan Restaurant', 'Video Game Store',
       'Vietnamese Restaurant', 'Warehouse Store', 'Wine Bar', 'Wine Shop',
       'Women's Store', 'Yoga Studio', 'Zoo', 'Zoo Exhibit'],
      dtype='object', length=186)

In [189]:
Camden_grouped.shape

(17, 186)

<h4>Printing each neighborhood with the top 5 most common venues <h4>

In [190]:

num_top_venues = 5
for hood in Camden_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Camden_grouped[Camden_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Belsize Park----
                venue  freq
0                 Pub  0.30
1   Indian Restaurant  0.07
2              Bakery  0.07
3         Pizza Place  0.04
4  Seafood Restaurant  0.04


----Bloomsbury----
                venue  freq
0                Café  0.18
1               Hotel  0.06
2  Italian Restaurant  0.06
3           Hotel Bar  0.06
4   Convenience Store  0.06


----Camden Town----
                venue  freq
0                 Pub  0.11
1               Hotel  0.10
2         Coffee Shop  0.08
3                Café  0.05
4  Italian Restaurant  0.05


----Chalk Farm----
                venue  freq
0                 Pub  0.18
1                Café  0.09
2                Park  0.09
3  Italian Restaurant  0.09
4       Garden Center  0.09


----Fitzrovia----
                venue  freq
0                Café  0.10
1                 Bar  0.08
2                 Pub  0.06
3  Italian Restaurant  0.05
4         Pizza Place  0.04


----Frognal----
                 venue  freq
0       

<h4>Sorting Venues in descending order <h4>

In [191]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [192]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Camden_grouped['Neighborhood']

for ind in np.arange(Camden_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Camden_grouped.iloc[ind, :], num_top_venues)

In [193]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belsize Park,Pub,Bakery,Indian Restaurant,Plaza,Park,Deli / Bodega,Coffee Shop,Seafood Restaurant,Café,Memorial Site
1,Bloomsbury,Café,Hotel,Hotel Bar,Italian Restaurant,Pub,Convenience Store,Outdoors & Recreation,Bus Stop,Coffee Shop,Deli / Bodega
2,Camden Town,Pub,Hotel,Coffee Shop,Italian Restaurant,Café,Bookstore,Burger Joint,Garden,Bakery,History Museum
3,Chalk Farm,Pub,Café,Park,Italian Restaurant,Garden Center,Pizza Place,Caribbean Restaurant,Event Space,Rental Car Location,Beer Bar
4,Fitzrovia,Café,Bar,Pub,Italian Restaurant,Pizza Place,Coffee Shop,Clothing Store,Bakery,Breakfast Spot,French Restaurant


<h4> Run K-means algorithm to cluster the neighborhood into 5 clusters <h4>

In [194]:
Camden_grouped.head()

,Neighborhood,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Belsize Park,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,...,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
1,Bloomsbury,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,...,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
2,Camden Town,0.0,0.0,0.0,0.0,0.0,0.0100,0.0,0.0000,0.0,...,0.0000,0.0,0.02,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
3,Chalk Farm,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,...,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
4,Fitzrovia,0.0,0.0,0.0,0.0,0.0,0.0125,0.0,0.0125,0.0,...,0.0125,0.0,0.00,0.0,0.0,0.0,0.0,0.0125,0.0,0.0


In [195]:
# set number of clusters
kclusters = 5
Camden_grouped_clustering = Camden_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Camden_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 0, 4, 1, 1, 1, 1, 1, 2], dtype=int32)

In [196]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

<h4>Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood<h4>

In [197]:
Camden_merged = Camden_borough
# add clustering labels
Camden_merged['Cluster Labels'] = kmeans.labels_

# merge Neighborhoods dataframe with Camden borough dataframe to add latitude/longitude for each neighborhood
Camden_merged = Camden_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Camden_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Camden,Belsize Park,51.571734,-0.147219,2,Pub,Bakery,Indian Restaurant,Plaza,Park,Deli / Bodega,Coffee Shop,Seafood Restaurant,Café,Memorial Site
1,Camden,Bloomsbury,51.545045,-0.165609,1,Café,Hotel,Hotel Bar,Italian Restaurant,Pub,Convenience Store,Outdoors & Recreation,Bus Stop,Coffee Shop,Deli / Bodega
2,Camden,Camden Town,51.526342,-0.120229,0,Pub,Hotel,Coffee Shop,Italian Restaurant,Café,Bookstore,Burger Joint,Garden,Bakery,History Museum
3,Camden,Chalk Farm,51.544545,-0.133901,4,Pub,Café,Park,Italian Restaurant,Garden Center,Pizza Place,Caribbean Restaurant,Event Space,Rental Car Location,Beer Bar
4,Camden,Fitzrovia,51.543966,-0.154115,1,Café,Bar,Pub,Italian Restaurant,Pizza Place,Coffee Shop,Clothing Store,Bakery,Breakfast Spot,French Restaurant


<h4>Visualize the clusters by plotting a map<h4>

In [198]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
y = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(y)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Camden_merged['Latitude'], Camden_merged['Longitude'], Camden_merged['Neighborhood'], Camden_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4> Examining the Clusters <h4>

In [117]:
Camden_merged.loc[Camden_merged['Cluster Labels'] == 0, Camden_merged.columns[[1] + list(range(5, Camden_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Camden Town,Pub,Hotel,Coffee Shop,Italian Restaurant,Café,Bookstore,Burger Joint,Garden,Bakery,History Museum
10,Kentish Town,Pub,Sandwich Place,Hotel,Café,Theater,Coffee Shop,Restaurant,Bookstore,History Museum,Japanese Restaurant
13,St Giles,Coffee Shop,Gym / Fitness Center,Café,Gourmet Shop,Italian Restaurant,Pub,Theater,Park,Plaza,Burger Joint
14,St Pancras,Pub,Sandwich Place,Hotel,Café,Theater,Coffee Shop,Restaurant,Bookstore,History Museum,Japanese Restaurant
15,Swiss Cottage,Pub,Hotel,Coffee Shop,Italian Restaurant,Café,Bookstore,Burger Joint,Garden,Bakery,History Museum


In [118]:
Camden_merged.loc[Camden_merged['Cluster Labels'] == 1, Camden_merged.columns[[1] + list(range(5, Camden_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bloomsbury,Café,Hotel,Hotel Bar,Italian Restaurant,Pub,Convenience Store,Outdoors & Recreation,Bus Stop,Coffee Shop,Deli / Bodega
4,Fitzrovia,Café,Bar,Pub,Italian Restaurant,Pizza Place,Coffee Shop,Clothing Store,Bakery,Breakfast Spot,French Restaurant
5,Frognal,Coffee Shop,Italian Restaurant,Cocktail Bar,Hotel,Café,Pizza Place,Japanese Restaurant,French Restaurant,Hotel Bar,Seafood Restaurant
6,Gospel Oak,Bakery,Café,Pub,Indian Restaurant,Breakfast Spot,Community Center,English Restaurant,Fruit & Vegetable Store,Bookstore,Coffee Shop
7,Hampstead,Gym / Fitness Center,Pub,Pool,Café,Music Venue,French Restaurant,Record Shop,Thai Restaurant,Gastropub,Coffee Shop
8,Highgate,Bakery,Café,Italian Restaurant,Pub,Pizza Place,Ice Cream Shop,Museum,Bookstore,Bistro,Thai Restaurant
11,Primrose Hill,Coffee Shop,Café,Pub,Market,Pizza Place,Flea Market,Street Food Gathering,Cocktail Bar,Bike Shop,French Restaurant
16,West Hampstead,Italian Restaurant,Grocery Store,Coffee Shop,Café,Hotel Bar,Chinese Restaurant,Pharmacy,Bakery,Hotel,Dessert Shop


In [119]:
Camden_merged.loc[Camden_merged['Cluster Labels'] == 2, Camden_merged.columns[[1] + list(range(5, Camden_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belsize Park,Pub,Bakery,Indian Restaurant,Plaza,Park,Deli / Bodega,Coffee Shop,Seafood Restaurant,Café,Memorial Site
9,Holborn,Pub,Bakery,Indian Restaurant,Plaza,Park,Deli / Bodega,Coffee Shop,Seafood Restaurant,Café,Memorial Site


In [120]:
Camden_merged.loc[Camden_merged['Cluster Labels'] == 3, Camden_merged.columns[[1] + list(range(5, Camden_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Somerstown,Zoo Exhibit,Pub,Italian Restaurant,Park,Coffee Shop,Café,Bar,French Restaurant,Convenience Store,Playground


In [121]:
Camden_merged.loc[Camden_merged['Cluster Labels'] == 4, Camden_merged.columns[[1] + list(range(5, Camden_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Chalk Farm,Pub,Café,Park,Italian Restaurant,Garden Center,Pizza Place,Caribbean Restaurant,Event Space,Rental Car Location,Beer Bar


<h3> Cluster Analysis and Conclusion <h3>

<h3>After careful and detailed analysis of the data from the neighborhoods, it has been identified that the neighborhoods of Camden Town, Kentish Town,St. Pancras, Swiss Cottage, Belsize Park, Holborn and Chalk Farm have 'Pubs' as the most visited venue.
Since Camden borough is very popular among the youngsters and promotes punk culture, I have been successful in identifying a new business opportunity ,i.e. opening of a PUB.
The neighborhood I have narrowed down to and identified is Camden Park since it is one of the most popular neighborhoods of Camden and of London city.Camden has the seventh largest economy in the UK. The area's industrial economic base has been replaced by service industries such as retail, tourism and entertainment. The area now hosts street markets and music venues that are strongly associated with alternative culture.<h3>